In [3]:
import fundus_image_toolbox as fit
import cv2
import os

dataset_root = "datasets/eye_cropped"        # input root
output_root = "datasets/eye_cropped_mask"    # output root
valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

# Load the segmentation ensemble (uses GPU if available)
ensemble = fit.load_segmentation_ensemble(device="cuda:0")

for dirpath, _, filenames in os.walk(dataset_root):
    image_files = [
        f for f in sorted(filenames)
        if os.path.splitext(f)[1].lower() in valid_ext
    ]
    if not image_files:
        continue

    # Mirror folder structure under output_root
    rel_dir = os.path.relpath(dirpath, dataset_root)
    out_dir = os.path.join(output_root, rel_dir)
    os.makedirs(out_dir, exist_ok=True)

    img_paths = [os.path.join(dirpath, f) for f in image_files]

    vessel_masks = fit.ensemble_predict_segmentation(
        ensemble,
        img_paths,
        threshold=0.5,
        size=(1024, 1024)
    )

    for img_file, vessel_mask in zip(image_files, vessel_masks):
        stem = os.path.splitext(img_file)[0]
        out_path = os.path.join(out_dir, f"{stem}_mask.png")
        cv2.imwrite(out_path, (vessel_mask * 255).astype("uint8"))

print(f"Done. Masks saved under: {output_root}")


Done. Masks saved under: datasets/eye_cropped_mask
